Решение соревнования https://cleancodecup.ru/problem/promopurchase

Посвящено предсказанию того, воспользуется ли человек промокодом. Данное решение пилотное, предсказывается всего лишь по 8 признакам, которые взяты с датасета с покупками, остальные датасеты пока не применялись.

In [113]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_log_error, accuracy_score
import optuna
import xgboost as xgb

In [55]:
cls_oct = pd.read_csv('clients_promo_october.csv')
mobile_events = pd.read_csv('mobile_events.csv')
orders = pd.read_csv('orders.csv')
train = pd.read_csv('train_target.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,ClientUUId,Id,OrderType,LocalBeginDate,LocalEndDate,OrderPrice,Discount,apply_promo
0,000D3A20F23EA95811E7B99F3ED09FC8,7,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,699,200,0
1,000D3A20F23EA95811E7B99F3ED09FC8,6,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,25,0
2,000D3A20F23EA95811E7BD373E79565E,5,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,799,200,0
3,000D3A20F23EA95811E7BD373E79565E,5,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,250,0
4,000D3A20F23EA95811E7BDAE71CD3F33,6,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,699,30,0


In [19]:
lst = set(list(train.ClientUUId)) - set(list(orders.ClientUUId))
lst = list(lst)

In [62]:
new_f = pd.merge(orders[['OrderUUId', 'ClientUUId']].drop_duplicates(), orders.groupby(['OrderUUId'])['ProductTotalPrice'].sum(), on='OrderUUId').drop(['OrderUUId'], axis=1)
new_f = pd.merge(new_f.groupby('ClientUUId').mean().reset_index(), new_f.groupby('ClientUUId').count().reset_index(), on='ClientUUId')
new_f_2 = pd.DataFrame(np.hstack((np.array(lst).reshape(-1, 1), np.zeros((len(lst), 2)))), columns=['ClientUUId', 'ProductTotalPrice_x', 'ProductTotalPrice_y'])
new_f = pd.concat((new_f, new_f_2), axis=0).reset_index(drop=True)
new_f.iloc[:, 1:] = new_f.iloc[:, 1:].astype(float).astype(int)
new_f_2 = pd.concat((orders.groupby('ClientUUId')['apply_promo'].sum().reset_index(), pd.concat((new_f_2.ClientUUId, pd.DataFrame(np.zeros(len(lst), dtype=int), columns=['apply_promo'])), axis=1)), axis=0).reset_index(drop=True)
new_f = pd.concat((new_f, new_f_2.apply_promo), axis=1)

In [37]:
new_f.head()

,ClientUUId,ProductTotalPrice_x,ProductTotalPrice_y
0,000D3A20F23EA95811E7B7104AC23CD3,1108,10
1,000D3A20F23EA95811E7B88F5E50100B,1002,9
2,000D3A20F23EA95811E7B8BD71C73877,857,10
3,000D3A20F23EA95811E7B95FE7E6321C,835,10
4,000D3A20F23EA95811E7B969B5BD2C3F,1122,10


In [96]:
new_train = train.drop(['LocalBeginDate', 'LocalEndDate'], axis=1)
new_train = pd.get_dummies(new_train, columns=['Id', 'OrderType'], dtype=int)

In [97]:
y = new_train.apply_promo
new_train = new_train.drop(['apply_promo'], axis=1)

In [100]:
X = pd.merge(new_train, new_f, on='ClientUUId', how='left')

In [101]:
X.drop(['ClientUUId'], axis=1, inplace=True)

In [102]:
X1, X2, y1, y2 = train_test_split(X.to_numpy().astype(int), y.to_numpy().astype(int), test_size=0.2, random_state=14)

In [108]:
model = XGBClassifier()
X1, y1 = RandomUnderSampler().fit_resample(X1, y1)
model.fit(X1, y1)

XGBClassifier()

In [109]:
y2_pred = model.predict(X2)

In [110]:
roc_auc_score(y2, y2_pred)

0.7209740018076453

In [111]:
confusion_matrix(y2, y2_pred)

array([[4064, 1310],
       [  55,  120]], dtype=int64)

In [123]:
def objective(trial):

    params = {
    'objective': 'reg:squarederror',
    'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
    'max_depth': trial.suggest_int('max_depth', 3, 10),
    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    'gamma': trial.suggest_float('gamma', 0, 5),
    'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
    'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
    }

    model = xgb.train(params, X_train, num_boost_round=trial.suggest_int('n_estimators', 100, 1000))

    y_pred = model.predict(X_test)

    return roc_auc_score(y2, y_pred)

In [124]:
X1, X2, y1, y2 = train_test_split(X.to_numpy().astype(int), y.to_numpy().astype(int), test_size=0.2, random_state=14)

In [ ]:
scal = StandardScaler()
X1 = scal.fit_transform(X1)
X_train = xgb.DMatrix(X1, label=y1)
X_test = xgb.DMatrix(X2, label=y2)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best Parameters:', study.best_params)
print('Best ROC_AUC:', study.best_value)

Trial 21 finished with value: 0.7630240842150035 and parameters: {'learning_rate': 0.0721584366956849, 'max_depth': 5, 'subsample': 0.5891893757533981, 'colsample_bytree': 0.81349156603394, 'gamma': 0.5461995811665656, 'reg_alpha': 0.701904559076098, 'reg_lambda': 1.7980227501439583, 'n_estimators': 479}. Best is trial 21 with value: 0.7630240842150035.

In [128]:
params = {'learning_rate': 0.0721584366956849, 'max_depth': 5, 'subsample': 0.5891893757533981, 'colsample_bytree': 0.81349156603394, 'gamma': 0.5461995811665656, 'reg_alpha': 0.701904559076098, 'reg_lambda': 1.7980227501439583, 'n_estimators': 479}

xgb_ = XGBClassifier(**params)

In [131]:
X, y = RandomUnderSampler().fit_resample(X.astype(int), y.astype(int))
xgb_.fit(X, y)

XGBClassifier(colsample_bytree=0.81349156603394, gamma=0.5461995811665656,
              learning_rate=0.0721584366956849, max_depth=5, n_estimators=479,
              reg_alpha=0.701904559076098, reg_lambda=1.7980227501439583,
              subsample=0.5891893757533981)

In [133]:
test.head()

,ClientUUId,Id,OrderType,LocalBeginDate,LocalEndDate,OrderPrice,Discount
0,000D3A20F23EA95811E7C0A95563344E,7,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200
1,000D3A20F23EA95811E7C7892A0CE261,5,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,699,200
2,000D3A20F23EA95811E7CD686C396528,6,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,20
3,000D3A20F23EA95911E7CEA8C574EDAE,5,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200
4,000D3A20F23EA95911E7D4F05C59C978,7,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200


In [134]:
lst = set(list(test.ClientUUId)) - set(list(orders.ClientUUId))
lst = list(lst)
new_f = pd.merge(orders[['OrderUUId', 'ClientUUId']].drop_duplicates(), orders.groupby(['OrderUUId'])['ProductTotalPrice'].sum(), on='OrderUUId').drop(['OrderUUId'], axis=1)
new_f = pd.merge(new_f.groupby('ClientUUId').mean().reset_index(), new_f.groupby('ClientUUId').count().reset_index(), on='ClientUUId')
new_f_2 = pd.DataFrame(np.hstack((np.array(lst).reshape(-1, 1), np.zeros((len(lst), 2)))), columns=['ClientUUId', 'ProductTotalPrice_x', 'ProductTotalPrice_y'])
new_f = pd.concat((new_f, new_f_2), axis=0).reset_index(drop=True)
new_f.iloc[:, 1:] = new_f.iloc[:, 1:].astype(float).astype(int)
new_f_2 = pd.concat((orders.groupby('ClientUUId')['apply_promo'].sum().reset_index(), pd.concat((new_f_2.ClientUUId, pd.DataFrame(np.zeros(len(lst), dtype=int), columns=['apply_promo'])), axis=1)), axis=0).reset_index(drop=True)
new_f = pd.concat((new_f, new_f_2.apply_promo), axis=1)

In [139]:
np.isin(test.ClientUUId, new_f.ClientUUId.unique()).min()

True

In [140]:
new_test = test.drop(['LocalBeginDate', 'LocalEndDate'], axis=1)
new_test = pd.get_dummies(new_test, columns=['Id', 'OrderType'], dtype=int)

In [142]:
X_test = pd.merge(new_test, new_f, on='ClientUUId', how='left')

In [145]:
X_test = X_test.drop(['ClientUUId'], axis=1).to_numpy()

In [150]:
pred_test = xgb_.predict_proba(X_test)[:, 1]

In [151]:
result = pd.concat((pd.read_csv('test.csv'), pd.DataFrame(pred_test, columns=['apply_promo'])), axis=1)

In [153]:
result.to_csv('submission.csv', index=False)

Результирующая метрика = 63492608

18 место